In [ ]:
%matplotlib inline


# Estimate a probability with FORM


In this example we estimate a failure probability with the `FORM` algorithm on the cantilever beam example. More precisely, we show how to use the associated results:

- the design point in both physical and standard space,
- the probability estimation according to the FORM approximation, and the following SORM ones: Tvedt, Hohen-Bichler and Breitung,
- the Hasofer reliability index and the generalized ones evaluated from the Breitung, Tvedt and Hohen-Bichler approximations,
- the importance factors defined as the normalized director factors of the design point in the $U$-space
- the sensitivity factors of the Hasofer reliability index and the FORM probability.
- the coordinates of the mean point in the standard event space.

The coordinates of the mean point in the standard event space is:

\begin{align}\frac{1}{E_1(-\beta)}\int_{\beta}^{\infty} u_1 p_1(u_1)du_1\end{align}


where $E_1$ is the spheric univariate distribution of the standard space and $\beta$ is the reliability index.



## Introduction 

Let us consider the analytical example of a cantilever beam with Young modulus E, length L and section modulus I.

One end of the cantilever beam is built in a wall and we apply a concentrated bending load F at the other end of the beam, resulting in a deviation:

\begin{align}d = \frac{FL^3}{3EI}\end{align}


Failure occurs when the beam deviation is too large:

\begin{align}d \ge 30 (cm)\end{align}


Four independent random variables are considered:

 - E: Young's modulus [Pa]
 - F: load [N]
 - L: length [m]
 - I: section [m^4]

Stochastic model (simplified model, no units):

 - E ~ Beta(0.93, 2.27, 2.8e7, 4.8e7)
 - F ~ LogNormal(30000, 9000, 15000)
 - L ~ Uniform(250, 260)
 - I ~ Beta(2.5, 1.5, 3.1e2, 4.5e2)




In [ ]:
from __future__ import print_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
ot.Log.Show(ot.Log.NONE)

Create the marginal distributions of the parameters.



In [ ]:
dist_E = ot.Beta(0.93, 2.27, 2.8e7, 4.8e7)
dist_F = ot.LogNormalMuSigma(30000, 9000, 15000).getDistribution()
dist_L = ot.Uniform(250, 260)
dist_I = ot.Beta(2.5, 1.5, 3.1e2, 4.5e2)
marginals = [dist_E, dist_F, dist_L, dist_I]

Create the Copula.



In [ ]:
RS = ot.CorrelationMatrix(4)
RS[2, 3] = -0.2
# Evaluate the correlation matrix of the Normal copula from RS
R = ot.NormalCopula.GetCorrelationFromSpearmanCorrelation(RS)
# Create the Normal copula parametrized by R
copula = ot.NormalCopula(R)

Create the joint probability distribution.



In [ ]:
distribution = ot.ComposedDistribution(marginals, copula)
distribution.setDescription(['E', 'F', 'L', 'I'])

create the model



In [ ]:
model = ot.SymbolicFunction(['E', 'F', 'L', 'I'], ['F*L^3/(3*E*I)'])

Create the event whose probability we want to estimate.



In [ ]:
vect = ot.RandomVector(distribution)
G = ot.CompositeRandomVector(model, vect)
event = ot.ThresholdEvent(G, ot.Greater(), 30.0)
event.setName("deviation")

Define a solver



In [ ]:
optimAlgo = ot.Cobyla()
optimAlgo.setMaximumEvaluationNumber(1000)
optimAlgo.setMaximumAbsoluteError(1.0e-10)
optimAlgo.setMaximumRelativeError(1.0e-10)
optimAlgo.setMaximumResidualError(1.0e-10)
optimAlgo.setMaximumConstraintError(1.0e-10)

Run FORM



In [ ]:
algo = ot.FORM(optimAlgo, event, distribution.getMean())
algo.run()
result = algo.getResult()

Probability



In [ ]:
result.getEventProbability()

Hasofer reliability index



In [ ]:
result.getHasoferReliabilityIndex()

Design point in the standard U* space.



In [ ]:
result.getStandardSpaceDesignPoint()

Design point in the physical X space.



In [ ]:
result.getPhysicalSpaceDesignPoint()

Importance factors



In [ ]:
graph = result.drawImportanceFactors()
view = viewer.View(graph)

In [ ]:
marginalSensitivity, otherSensitivity = result.drawHasoferReliabilityIndexSensitivity()
marginalSensitivity.setLegendPosition('bottom')
view = viewer.View(marginalSensitivity)

In [ ]:
marginalSensitivity, otherSensitivity = result.drawEventProbabilitySensitivity()
view = viewer.View(marginalSensitivity)

Error history



In [ ]:
optimResult = result.getOptimizationResult()
graphErrors = optimResult.drawErrorHistory()
graphErrors.setLegendPosition('bottom')
graphErrors.setYMargin(0.0)
view = viewer.View(graphErrors)

Get additional results with SORM



In [ ]:
algo = ot.SORM(optimAlgo, event, distribution.getMean())
algo.run()
sorm_result = algo.getResult()

Reliability index with Breitung approximation



In [ ]:
sorm_result.getGeneralisedReliabilityIndexBreitung()

... with Hohenbichler approximation



In [ ]:
sorm_result.getGeneralisedReliabilityIndexHohenbichler()

.. with Tvedt approximation



In [ ]:
sorm_result.getGeneralisedReliabilityIndexTvedt()

SORM probability of the event with Breitung approximation



In [ ]:
sorm_result.getEventProbabilityBreitung()

... with Hohenbichler approximation



In [ ]:
sorm_result.getEventProbabilityHohenbichler()

... with Tvedt approximation



In [ ]:
sorm_result.getEventProbabilityTvedt()

plt.show()